In [1]:
## TRABAJO PySpark

## Integrantes del equipo

## 1. María del Pilar Hernández Moreno CC 1152458223
## 2. Andrés Felipe Hernandez velandia CC 1143390458
## 3. Gabriel Jaime Sánchez Montoya CC 1017229936
## 4. Rubén Esneyder Morales Barbosa CC 1072429901

In [2]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [3]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell.
!hadoop fs -put Asistencia_del_Pitch_2021.04.13_ISoftware.csv
!hadoop fs -put Evaluacion_del_Pitch_2021.04.13_ISoftware.csv
!hadoop fs -ls

Found 2 items
-rw-r--r--   1 root supergroup      93121 2021-05-19 00:51 Asistencia_del_Pitch_2021.04.13_ISoftware.csv
-rw-r--r--   1 root supergroup     111944 2021-05-19 00:51 Evaluacion_del_Pitch_2021.04.13_ISoftware.csv


In [4]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales

ContextoSql.sql("""
CREATE TABLE IF NOT EXISTS 
Evaluacion (
            Marca_temporal STRING,
            Nombre_usuario STRING,
            Equipo STRING,
            Introduccion INT,
            Equipo_nota INT,
            Problema INT,
            Ventajas INT,
            Solucion INT,
            Producto INT,
            Traccion INT,
            Mercado INT,
            Competencia INT,
            Modelo INT,
            Inversion INT,
            Contacto INT,
            Exposicion_1 INT,
            Exposicion_2 INT,
            Exposicion_3 INT,
            Decision STRING,
            Observaciones STRING
            ) 
USING CSV OPTIONS ( 
    header='true', 
    nullvalue='NA', 
    timestampFormat=\"yyyy-MM-dd'T'HH:mm:ss\", 
    path='Evaluacion_del_Pitch_2021.04.13_ISoftware.csv')
    """);

ContextoSql.sql("""
CREATE TABLE IF NOT EXISTS 
Asistencia (
            Marca_temporal STRING,
            Nombre_usuario STRING,
            Equipo_pertenece STRING,
            Equipo_expone STRING
            ) 
USING CSV OPTIONS ( 
    header='true', 
    nullvalue='NA', 
    timestampFormat=\"yyyy-MM-dd'T'HH:mm:ss\", 
    path='Asistencia_del_Pitch_2021.04.13_ISoftware.csv')
    """);

In [6]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente

consulta = ContextoSql.sql("""
SELECT DISTINCT Nombre_usuario 
FROM Asistencia 
ORDER BY Nombre_usuario ASC
"""
)
consulta.show()
print("Se encontraron: ", consulta.count()," registros.")

+--------------------+
|      Nombre_usuario|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
|cquinchiar@unal.e...|
| dadazam@unal.edu.co|
|daestradam@unal.e...|
|davgarciava@unal....|
|dballesteroso@una...|
|  dbrito@unal.edu.co|
|dcadavid@unal.edu.co|
|dcardonaal@unal.e...|
|dchavarriar@unal....|
|dgarciabl@unal.ed...|
+--------------------+
only showing top 20 rows

Se encontraron:  83  registros.


In [9]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia

consulta2 = ContextoSql.sql("""
SELECT Nombre_usuario as Nombre_usuario, count(Nombre_usuario) as Registros, min(replace(substring(Marca_temporal,1,length(Marca_temporal)-6)," ","")) as Minimo, max(replace(substring(Marca_temporal,1,length(Marca_temporal)-6)," ","")) as Maximo
FROM Asistencia
GROUP BY Nombre_usuario
ORDER BY Nombre_usuario
"""
)
consulta2.show()

+--------------------+---------+--------------------+--------------------+
|      Nombre_usuario|Registros|              Minimo|              Maximo|
+--------------------+---------+--------------------+--------------------+
| abanolc@unal.edu.co|        7|2021/04/08 8:12:1...|2021/04/13 9:37:5...|
|acastrillonv@unal...|        8|2021/04/08 8:13:0...|2021/04/13 9:43:1...|
| aflemag@unal.edu.co|        8|2021/04/08 8:11:5...|2021/04/13 9:41:5...|
|anapariciom@unal....|       10|2021/04/08 8:11:4...|2021/04/13 9:38:5...|
|angutierrezb@unal...|        8|2021/04/08 8:19:0...|2021/04/13 9:43:5...|
|anoriega@unal.edu.co|        8|2021/04/08 8:15:1...|2021/04/13 9:40:2...|
|auarbelaeza@unal....|        8|2021/04/08 8:09:5...|2021/04/13 9:41:0...|
| bocampo@unal.edu.co|        8|2021/04/08 8:13:3...|2021/04/13 9:42:1...|
|cgiraldo@unal.edu.co|        8|2021/04/08 8:44:2...|2021/04/13 9:56:4...|
|cjfunezg@unal.edu.co|        8|2021/04/08 8:12:1...|2021/04/13 9:41:4...|
|cquinchiar@unal.e...|   

In [10]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).

consulta3 = ContextoSql.sql("""
SELECT Nombre_usuario as Nombre_usuario, count(Nombre_usuario) as Registros 
FROM Asistencia
GROUP BY Nombre_usuario
ORDER BY Nombre_usuario
"""
)
registros = consulta3.filter(consulta3['Registros'] < '2')
registros.show()
print("Se encontraron: ", registros.count()," registros.")

+--------------------+---------+
|      Nombre_usuario|Registros|
+--------------------+---------+
|jdroldano@unal.ed...|        1|
+--------------------+---------+

Se encontraron:  1  registros.


In [11]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.

consulta4 = ContextoSql.sql("""
SELECT Nombre_usuario as Nombre_usuario, count(Nombre_usuario) as Registros 
FROM Asistencia
GROUP BY Nombre_usuario
ORDER BY Nombre_usuario
"""
)
datos = consulta4.filter(consulta4['Registros'] < '0')
datos.show()
print("Se encontraron: ", datos.count()," registros.")

+--------------+---------+
|Nombre_usuario|Registros|
+--------------+---------+
+--------------+---------+

Se encontraron:  0  registros.


In [12]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.

consulta5 = ContextoSql.sql("""
SELECT DISTINCT substring(Equipo_pertenece,15,length(Equipo_pertenece)) as Equipo, Nombre_usuario as Integrantes
FROM Asistencia
ORDER BY Equipo
"""
)
consulta5.show()

+--------------------+--------------------+
|              Equipo|         Integrantes|
+--------------------+--------------------+
|Administración de...|emflorezb@unal.ed...|
|Administración de...|angutierrezb@unal...|
|Administración de...| eapenad@unal.edu.co|
|Administración de...|lumarinb@unal.edu.co|
|Administración de...|jhcordoba@unal.ed...|
|Administración de...| kmolano@unal.edu.co|
|Administración de...|rmbuilesm@unal.ed...|
|Administración de...| jortize@unal.edu.co|
|Administración de...|nsalinas@unal.edu.co|
|Administración de...|sednarvaezna@unal...|
|Administración de...|saespinosab@unal....|
|Administración de...|framirezf@unal.ed...|
|Administración de...|samolinap@unal.ed...|
|Gestión de Menús ...|hriveraa@unal.edu.co|
|Gestión de Menús ...|emlopezr@unal.edu.co|
|Gestión de Menús ...| lrdiaza@unal.edu.co|
|Gestión de Menús ...|anapariciom@unal....|
|Gestión de Menús ...| abanolc@unal.edu.co|
|Gestión de Menús ...|jcardonaso@unal.e...|
|Gestión de Menús ...|favasquezj

In [13]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.

consulta6 = ContextoSql.sql("""
SELECT substring(Equipo_expone,8,length(Equipo_expone)) as Expone, count(*) as Cantidad
FROM Asistencia
WHERE substring(Equipo_pertenece,15,length(Equipo_pertenece)) != substring(Equipo_expone,9,length(Equipo_expone))
GROUP BY Expone
"""
)

consulta6.show()

+--------------------+--------+
|              Expone|Cantidad|
+--------------------+--------+
| Lo tengo y te lo...|      75|
| Gestión de Resid...|      72|
| Gestión de progr...|      74|
|     Mi Sanandresito|      72|
| Deserción de est...|       3|
| Servicios a domi...|      76|
| Administración d...|      70|
|       Te lo cambio.|      75|
| Gestión de Menús...|      72|
+--------------------+--------+



In [14]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, 
#pues un integrante no puede evaluar a su propio equipo.

consulta8 = ContextoSql.sql("""
SELECT DISTINCT e.Nombre_usuario, e.Equipo as Equipo_a_evaluar, substring(a.Equipo_pertenece,15,length(a.Equipo_pertenece)) AS Equipo_pertenece 
FROM Evaluacion e
LEFT JOIN
    (SELECT Nombre_usuario, Equipo_pertenece FROM Asistencia) a
ON
    e.Nombre_usuario= a.Nombre_usuario
WHERE Equipo= substring(Equipo_pertenece,15,length(Equipo_pertenece))
""")

consulta8.show()

+--------------------+--------------------+--------------------+
|      Nombre_usuario|    Equipo_a_evaluar|    Equipo_pertenece|
+--------------------+--------------------+--------------------+
|daestradam@unal.e...|Lo tengo y te lo ...|Lo tengo y te lo ...|
|jhcordoba@unal.ed...|Administración de...|Administración de...|
|hriveraa@unal.edu.co|Gestión de Menús ...|Gestión de Menús ...|
|mcarvajalsa@unal....|Gestión de Reside...|Gestión de Reside...|
| jruedat@unal.edu.co|Servicios a domic...|Servicios a domic...|
|anapariciom@unal....|Servicios a domic...|Servicios a domic...|
|  lmayar@unal.edu.co|Gestión de progra...|Gestión de progra...|
| kmolano@unal.edu.co|Administración de...|Administración de...|
+--------------------+--------------------+--------------------+



In [15]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. 
# Recuerde que no son válidas las evaluaciones realizadas por los miembros del mismo equipo.

consulta9 = ContextoSql.sql("""
SELECT  e.Equipo, AVG(Introduccion) as Introduccion, AVG(Equipo_nota) as Equipo_nota, AVG(Problema) as Problema, AVG(Ventajas) as Ventajas, AVG(Solucion) as Solucion, AVG(Producto) as Producto,
        AVG(Traccion) as Traccion, AVG(Mercado) as Mercado, AVG(Competencia) as Competencia, AVG(Modelo) as Modelo, AVG(Inversion) as Inversion, AVG(Contacto) as Contacto,
        AVG(Exposicion_1) as Exposicion_1, AVG(Exposicion_2) as Exposicion_2, AVG(Exposicion_3) as Exposicion_3,
        (AVG(Introduccion) + AVG(Equipo_nota) + AVG(Problema) + AVG(Ventajas) + AVG(Solucion) + AVG(Producto) + AVG(Traccion) + AVG(Mercado) + AVG(Competencia) + AVG(Modelo) + AVG(Inversion) + AVG(Contacto) + AVG(Exposicion_1) + AVG(Exposicion_2) + AVG(Exposicion_3))/15 as Nota_final
FROM Evaluacion e
LEFT JOIN
    (SELECT Nombre_usuario, Equipo_pertenece FROM Asistencia) a
ON
    e.Nombre_usuario= a.Nombre_usuario
WHERE Equipo != substring(Equipo_pertenece,15,length(Equipo_pertenece))
GROUP BY Equipo
"""
)

consulta9.show()

+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+
|              Equipo|      Introduccion|       Equipo_nota|          Problema|          Ventajas|          Solucion|          Producto|          Traccion|           Mercado|       Competencia|            Modelo|         Inversion|          Contacto|      Exposicion_1|      Exposicion_2|     Exposicion_3|       Nota_final|
+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+
|Servicios a domic...| 4.

In [47]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate 
# mostrar todos los empatados) y el mejor equipo según el promedio total.

consulta10 = ContextoSql.sql("""
WITH Promedios AS (
SELECT  e.Equipo, AVG(Introduccion) as Introduccion, AVG(Equipo_nota) as Equipo_nota, AVG(Problema) as Problema, AVG(Ventajas) as Ventajas, AVG(Solucion) as Solucion, AVG(Producto) as Producto,
        AVG(Traccion) as Traccion, AVG(Mercado) as Mercado, AVG(Competencia) as Competencia, AVG(Modelo) as Modelo, AVG(Inversion) as Inversion, AVG(Contacto) as Contacto,
        AVG(Exposicion_1) as Exposicion_1, AVG(Exposicion_2) as Exposicion_2, AVG(Exposicion_3) as Exposicion_3,
        (AVG(Introduccion) + AVG(Equipo_nota) + AVG(Problema) + AVG(Ventajas) + AVG(Solucion) + AVG(Producto) + AVG(Traccion) + AVG(Mercado) + AVG(Competencia) + AVG(Modelo) + AVG(Inversion) + AVG(Contacto) + AVG(Exposicion_1) + AVG(Exposicion_2) + AVG(Exposicion_3))/15 as Nota_final
FROM Evaluacion e
LEFT JOIN
    (SELECT Nombre_usuario, Equipo_pertenece FROM Asistencia) a
ON
    e.Nombre_usuario= a.Nombre_usuario
WHERE Equipo != substring(Equipo_pertenece,15,length(Equipo_pertenece))
GROUP BY Equipo
)
SELECT Equipo, "Introduccion" AS Item_a_evaluar
FROM Promedios
WHERE Introduccion = (SELECT MAX(Introduccion) FROM Promedios)

UNION ALL

SELECT Equipo, "Equipo_nota" AS Item_a_evaluar
FROM Promedios
WHERE Equipo_nota = (SELECT MAX(Equipo_nota) FROM Promedios)

UNION ALL

SELECT Equipo, "Problema" AS Item_a_evaluar
FROM Promedios
WHERE Problema = (SELECT MAX(Problema) FROM Promedios)

UNION ALL

SELECT Equipo, "Ventajas" AS Item_a_evaluar
FROM Promedios
WHERE Ventajas = (SELECT MAX(Ventajas) FROM Promedios)

UNION ALL

SELECT Equipo, "Solucion" AS Item_a_evaluar
FROM Promedios
WHERE Solucion = (SELECT MAX(Solucion) FROM Promedios)

UNION ALL

SELECT Equipo, "Producto" AS Item_a_evaluar
FROM Promedios
WHERE Producto = (SELECT MAX(Producto) FROM Promedios)

UNION ALL

SELECT Equipo, "Traccion" AS Item_a_evaluar
FROM Promedios
WHERE Traccion = (SELECT MAX(Traccion) FROM Promedios)

UNION ALL

SELECT Equipo, "Mercado" AS Item_a_evaluar
FROM Promedios
WHERE Mercado = (SELECT MAX(Mercado) FROM Promedios)

UNION ALL

SELECT Equipo, "Competencia" AS Item_a_evaluar
FROM Promedios
WHERE Competencia = (SELECT MAX(Competencia) FROM Promedios)

UNION ALL

SELECT Equipo, "Modelo" AS Item_a_evaluar
FROM Promedios
WHERE Modelo = (SELECT MAX(Modelo) FROM Promedios)

UNION ALL

SELECT Equipo, "Inversion" AS Item_a_evaluar
FROM Promedios
WHERE Inversion = (SELECT MAX(Inversion) FROM Promedios)

UNION ALL

SELECT Equipo, "Contacto" AS Item_a_evaluar
FROM Promedios
WHERE Contacto = (SELECT MAX(Contacto) FROM Promedios)

UNION ALL

SELECT Equipo, "Exposicion_1" AS Item_a_evaluar
FROM Promedios
WHERE Exposicion_1 = (SELECT MAX(Exposicion_1) FROM Promedios)

UNION ALL

SELECT Equipo, "Exposicion_2" AS Item_a_evaluar
FROM Promedios
WHERE Exposicion_2 = (SELECT MAX(Exposicion_2) FROM Promedios)

UNION ALL

SELECT Equipo, "Exposicion_3" AS Item_a_evaluar
FROM Promedios
WHERE Exposicion_3 = (SELECT MAX(Exposicion_3) FROM Promedios)

UNION ALL

SELECT Equipo, "Nota_final" AS Item_a_evaluar
FROM Promedios
WHERE Nota_final = (SELECT MAX(Nota_final) FROM Promedios)

"""
)

consulta10.show()

+--------------------+--------------+
|              Equipo|Item_a_evaluar|
+--------------------+--------------+
|Gestión de Reside...|  Introduccion|
|Deserción de estu...|   Equipo_nota|
|Deserción de estu...|      Problema|
|Deserción de estu...|      Ventajas|
|Servicios a domic...|      Solucion|
|Deserción de estu...|      Producto|
|Deserción de estu...|      Traccion|
|Gestión de Reside...|       Mercado|
|Deserción de estu...|   Competencia|
|Gestión de Menús ...|        Modelo|
|Gestión de progra...|     Inversion|
|Deserción de estu...|      Contacto|
|       Te lo cambio.|  Exposicion_1|
|Lo tengo y te lo ...|  Exposicion_2|
|Deserción de estu...|  Exposicion_3|
|Deserción de estu...|    Nota_final|
+--------------------+--------------+



In [48]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate 
# mostrar todos los empatados) y el peor equipo según el promedio total.

consulta11 = ContextoSql.sql("""
WITH Promedios AS (
SELECT  e.Equipo, AVG(Introduccion) as Introduccion, AVG(Equipo_nota) as Equipo_nota, AVG(Problema) as Problema, AVG(Ventajas) as Ventajas, AVG(Solucion) as Solucion, AVG(Producto) as Producto,
        AVG(Traccion) as Traccion, AVG(Mercado) as Mercado, AVG(Competencia) as Competencia, AVG(Modelo) as Modelo, AVG(Inversion) as Inversion, AVG(Contacto) as Contacto,
        AVG(Exposicion_1) as Exposicion_1, AVG(Exposicion_2) as Exposicion_2, AVG(Exposicion_3) as Exposicion_3,
        (AVG(Introduccion) + AVG(Equipo_nota) + AVG(Problema) + AVG(Ventajas) + AVG(Solucion) + AVG(Producto) + AVG(Traccion) + AVG(Mercado) + AVG(Competencia) + AVG(Modelo) + AVG(Inversion) + AVG(Contacto) + AVG(Exposicion_1) + AVG(Exposicion_2) + AVG(Exposicion_3))/15 as Nota_final
FROM Evaluacion e
LEFT JOIN
    (SELECT Nombre_usuario, Equipo_pertenece FROM Asistencia) a
ON
    e.Nombre_usuario= a.Nombre_usuario
WHERE Equipo != substring(Equipo_pertenece,15,length(Equipo_pertenece))
GROUP BY Equipo
)
SELECT Equipo, "Introduccion" AS Item_a_evaluar
FROM Promedios
WHERE Introduccion = (SELECT MIN(Introduccion) FROM Promedios)

UNION ALL

SELECT Equipo, "Equipo_nota" AS Item_a_evaluar
FROM Promedios
WHERE Equipo_nota = (SELECT MIN(Equipo_nota) FROM Promedios)

UNION ALL

SELECT Equipo, "Problema" AS Item_a_evaluar
FROM Promedios
WHERE Problema = (SELECT MIN(Problema) FROM Promedios)

UNION ALL

SELECT Equipo, "Ventajas" AS Item_a_evaluar
FROM Promedios
WHERE Ventajas = (SELECT MIN(Ventajas) FROM Promedios)

UNION ALL

SELECT Equipo, "Solucion" AS Item_a_evaluar
FROM Promedios
WHERE Solucion = (SELECT MIN(Solucion) FROM Promedios)

UNION ALL

SELECT Equipo, "Producto" AS Item_a_evaluar
FROM Promedios
WHERE Producto = (SELECT MIN(Producto) FROM Promedios)

UNION ALL

SELECT Equipo, "Traccion" AS Item_a_evaluar
FROM Promedios
WHERE Traccion = (SELECT MIN(Traccion) FROM Promedios)

UNION ALL

SELECT Equipo, "Mercado" AS Item_a_evaluar
FROM Promedios
WHERE Mercado = (SELECT MIN(Mercado) FROM Promedios)

UNION ALL

SELECT Equipo, "Competencia" AS Item_a_evaluar
FROM Promedios
WHERE Competencia = (SELECT MIN(Competencia) FROM Promedios)

UNION ALL

SELECT Equipo, "Modelo" AS Item_a_evaluar
FROM Promedios
WHERE Modelo = (SELECT MIN(Modelo) FROM Promedios)

UNION ALL

SELECT Equipo, "Inversion" AS Item_a_evaluar
FROM Promedios
WHERE Inversion = (SELECT MIN(Inversion) FROM Promedios)

UNION ALL

SELECT Equipo, "Contacto" AS Item_a_evaluar
FROM Promedios
WHERE Contacto = (SELECT MIN(Contacto) FROM Promedios)

UNION ALL

SELECT Equipo, "Exposicion_1" AS Item_a_evaluar
FROM Promedios
WHERE Exposicion_1 = (SELECT MIN(Exposicion_1) FROM Promedios)

UNION ALL

SELECT Equipo, "Exposicion_2" AS Item_a_evaluar
FROM Promedios
WHERE Exposicion_2 = (SELECT MIN(Exposicion_2) FROM Promedios)

UNION ALL

SELECT Equipo, "Exposicion_3" AS Item_a_evaluar
FROM Promedios
WHERE Exposicion_3 = (SELECT MIN(Exposicion_3) FROM Promedios)

UNION ALL

SELECT Equipo, "Nota_final" AS Item_a_evaluar
FROM Promedios
WHERE Nota_final = (SELECT MIN(Nota_final) FROM Promedios)

"""
)

consulta11.show()

+--------------------+--------------+
|              Equipo|Item_a_evaluar|
+--------------------+--------------+
|Administración de...|  Introduccion|
|     Mi Sanandresito|   Equipo_nota|
|Administración de...|      Problema|
|Gestión de progra...|      Ventajas|
|Administración de...|      Solucion|
|Gestión de Menús ...|      Producto|
|Administración de...|      Traccion|
|       Te lo cambio.|       Mercado|
|     Mi Sanandresito|   Competencia|
|Servicios a domic...|        Modelo|
|       Te lo cambio.|     Inversion|
|Administración de...|      Contacto|
|     Mi Sanandresito|  Exposicion_1|
|     Mi Sanandresito|  Exposicion_2|
|     Mi Sanandresito|  Exposicion_3|
|Administración de...|    Nota_final|
+--------------------+--------------+



In [60]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)

consulta12 = ContextoSql.sql("""
SELECT  A.Nombre_usuario, (AVG(Introduccion) + AVG(Equipo_nota) + AVG(Problema) + AVG(Ventajas) + AVG(Solucion) + AVG(Producto) + AVG(Traccion) + AVG(Mercado) + AVG(Competencia) + AVG(Modelo) + AVG(Inversion) + AVG(Contacto) + AVG(Exposicion_1) + AVG(Exposicion_2) + AVG(Exposicion_3))/15 as Nota_final
FROM Evaluacion e
RIGHT JOIN Asistencia A ON e.Equipo = substring(A.Equipo_pertenece,15,length(A.Equipo_pertenece))
GROUP BY A.Nombre_usuario
"""
)

consulta12.show()


+--------------------+-----------------+
|      Nombre_usuario|       Nota_final|
+--------------------+-----------------+
|samolinap@unal.ed...|4.187619047619048|
| jruedat@unal.edu.co|4.424074074074074|
|nvalenciat@unal.e...|4.285446009389672|
|serendona@unal.ed...|              4.4|
|jgutierrezlo@unal...|4.285446009389672|
|dgerenal@unal.edu.co|4.285446009389672|
|cjfunezg@unal.edu.co|4.424074074074074|
|emflorezb@unal.ed...|4.187619047619048|
|dchavarriar@unal....|              4.4|
|fguerrerot@unal.e...|4.285446009389672|
|jdroldano@unal.ed...|4.424074074074074|
|juriveras@unal.ed...|4.470531400966183|
|jovillarrealm@una...|4.202739726027397|
|dguardia@unal.edu.co|4.470531400966183|
| lrdiaza@unal.edu.co|4.355251141552512|
|fmiranda@unal.edu.co|4.202739726027397|
|dcadavid@unal.edu.co|4.202739726027397|
| bocampo@unal.edu.co|4.424074074074074|
|rmbuilesm@unal.ed...|4.187619047619048|
|jmunozhe@unal.edu.co| 4.47699530516432|
+--------------------+-----------------+
only showing top